In [64]:
import numpy as np
import pandas as pd
import scipy as sp
import math
from scipy.optimize import minimize

ImportError: cannot import name 'maximize' from 'scipy.optimize' (C:\Users\Arjun Kejriwal\anaconda3\lib\site-packages\scipy\optimize\__init__.py)

In [4]:
# Reading in the asset prices

asset_prices = pd.read_csv("Training Data_Case 3.csv")
asset_prices = asset_prices.drop(columns = asset_prices.columns[0])
asset_prices

,A,B,C,D,E,F,G,H,I,J
0,119.96,198.25,76.54,32.08,53.20,19.47,40.63,43.47,19.76,113.91
1,120.32,201.42,76.73,31.86,53.88,19.23,40.47,45.16,19.88,112.98
2,119.60,200.23,77.60,31.56,55.11,18.93,39.84,46.03,20.29,114.24
3,121.76,199.61,79.48,31.62,55.80,19.15,40.13,46.89,20.01,114.27
4,120.10,199.64,81.49,31.89,55.02,19.16,39.94,47.59,20.12,113.87
...,...,...,...,...,...,...,...,...,...,...
2515,317.31,1385.04,199.14,273.02,52.11,89.45,151.54,390.13,88.25,35.99
2516,317.20,1358.01,202.59,277.45,52.73,90.93,152.39,394.03,87.56,35.89
2517,312.31,1315.00,198.29,281.23,52.71,92.28,153.92,375.70,89.54,36.13
2518,312.45,1319.59,200.78,282.77,52.26,93.49,155.26,368.33,91.36,35.15


In [5]:
# Creating the return dataset

returns = pd.DataFrame(columns=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'])

for i in range(0, 2519):
    returns.at[i,'A'] = ((asset_prices.at[i+1,'A'] - asset_prices.at[i,'A']) / asset_prices.at[i,'A'])*100
    returns.at[i,'B'] = ((asset_prices.at[i+1,'B'] - asset_prices.at[i,'B']) / asset_prices.at[i,'B'])*100
    returns.at[i,'C'] = ((asset_prices.at[i+1,'C'] - asset_prices.at[i,'C']) / asset_prices.at[i,'C'])*100
    returns.at[i,'D'] = ((asset_prices.at[i+1,'D'] - asset_prices.at[i,'D']) / asset_prices.at[i,'D'])*100
    returns.at[i,'E'] = ((asset_prices.at[i+1,'E'] - asset_prices.at[i,'E']) / asset_prices.at[i,'E'])*100
    returns.at[i,'F'] = ((asset_prices.at[i+1,'F'] - asset_prices.at[i,'F']) / asset_prices.at[i,'F'])*100
    returns.at[i,'G'] = ((asset_prices.at[i+1,'G'] - asset_prices.at[i,'G']) / asset_prices.at[i,'G'])*100
    returns.at[i,'H'] = ((asset_prices.at[i+1,'H'] - asset_prices.at[i,'H']) / asset_prices.at[i,'H'])*100
    returns.at[i,'I'] = ((asset_prices.at[i+1,'I'] - asset_prices.at[i,'I']) / asset_prices.at[i,'I'])*100
    returns.at[i,'J'] = ((asset_prices.at[i+1,'J'] - asset_prices.at[i,'J']) / asset_prices.at[i,'J'])*100

returns

,A,B,C,D,E,F,G,H,I,J
0,0.3001,1.598991,0.248236,-0.685786,1.278195,-1.232666,-0.393798,3.887739,0.607287,-0.816434
1,-0.598404,-0.590805,1.133846,-0.94162,2.282851,-1.560062,-1.556709,1.926484,2.062374,1.115242
2,1.80602,-0.309644,2.42268,0.190114,1.252041,1.162176,0.727912,1.868347,-1.37999,0.026261
3,-1.363338,0.015029,2.528938,0.85389,-1.397849,0.052219,-0.473461,1.492856,0.549725,-0.350048
4,-1.998335,-0.731316,-0.490858,-0.972092,4.125772,-1.878914,-1.22684,0.0,0.894632,0.184421
...,...,...,...,...,...,...,...,...,...,...
2514,3.331379,2.260746,1.992318,3.115912,-0.420409,5.371657,6.725826,1.429946,1.776035,1.810467
2515,-0.034666,-1.951568,1.73245,1.622592,1.189791,1.654556,0.560908,0.999667,-0.78187,-0.277855
2516,-1.541614,-3.167134,-2.122513,1.362408,-0.037929,1.484659,1.004003,-4.65193,2.261307,0.66871
2517,0.044827,0.349049,1.255737,0.547594,-0.853728,1.311227,0.870582,-1.961672,2.032611,-2.712427


In [23]:
# Converting the type of  

stocks = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']

for s in stocks:
    returns.loc[:, s] = pd.to_numeric(returns.loc[:,s])

returns.dtypes

A    float64
B    float64
C    float64
D    float64
E    float64
F    float64
G    float64
H    float64
I    float64
J    float64
dtype: object

In [25]:
# Calculating the covariance-variance matrix

cov_matrix = pd.DataFrame(np.cov(np.transpose(returns)))
cov_matrix

,0,1,2,3,4,5,6,7,8,9
0,3.230972,0.934087,-0.032461,2.674093,0.119448,3.620905,3.027342,0.041670,0.754096,0.096986
1,0.934087,2.003063,0.361989,0.794932,-0.147175,0.648404,0.809582,-0.016062,0.824611,-0.278279
2,-0.032461,0.361989,1.712451,-0.161588,0.688832,-0.394596,-0.218290,-0.061627,0.154809,0.816653
3,2.674093,0.794932,-0.161588,3.644457,0.209393,4.045706,3.340078,0.048285,0.659194,0.167932
4,0.119448,-0.147175,0.688832,0.209393,3.457709,0.460334,0.297555,-0.003426,-0.163309,2.864840
5,3.620905,0.648404,-0.394596,4.045706,0.460334,6.990791,4.704607,0.061810,0.595844,0.503963
6,3.027342,0.809582,-0.218290,3.340078,0.297555,4.704607,4.264499,0.110400,0.699143,0.304505
7,0.041670,-0.016062,-0.061627,0.048285,-0.003426,0.061810,0.110400,8.721751,-0.013486,0.135209
8,0.754096,0.824611,0.154809,0.659194,-0.163309,0.595844,0.699143,-0.013486,2.026851,-0.205436
9,0.096986,-0.278279,0.816653,0.167932,2.864840,0.503963,0.304505,0.135209,-0.205436,5.383232


In [69]:
# Running optimization on the first day returns 

initial_weights = [0.1 for i in range(0, 10)]
first_return = returns.loc[0, :]
initial_weights = np.array(initial_weights)
first_return = np.array(first_return)

def portfolio_objective(weights, returns, cov_matrix):
    portfolio_return = np.dot(weights, first_return)
    portfolio_risk = np.matmul(np.matmul(weights, cov_matrix), np.transpose(weights))
    return -(portfolio_return/math.sqrt(portfolio_risk))

constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
               {'type': 'ineq', 'fun': lambda x: x}]

result = minimize(portfolio_objective, initial_weights, args=(returns, cov_matrix), method='SLSQP', bounds=[(0, 1)] * returns.shape[1], constraints=constraints)
optimized_weights = result.x
print(optimized_weights)
print(result)

#for i in range(0, 2519):
    #daily_return = returns.loc[i,:]
    

'''
Things to improve:

1) Split into testing and training sets through an 80-20 split (thus your initial covariance matrix will only incorporate 80% of the data)
2) Extend the above solution to all 100% of the data, and create the functionality of reading the data UChicago will provide and calculating the optimal weights   

'''
    

[0.00000000e+00 4.93683920e-01 0.00000000e+00 2.36228715e-16
 2.40706316e-01 5.14030408e-16 6.50541008e-16 2.65609765e-01
 0.00000000e+00 3.60583905e-16]
     fun: -1.8941142908922206
     jac: array([ 4.83595327e-01,  4.77746129e-04,  2.70268753e-01,  1.29222909e+00,
       -2.92992592e-03,  1.76574220e+00,  1.09988876e+00,  1.76016986e-03,
        5.83638251e-03,  1.60439886e+00])
 message: 'Optimization terminated successfully'
    nfev: 104
     nit: 9
    njev: 9
  status: 0
 success: True
       x: array([0.00000000e+00, 4.93683920e-01, 0.00000000e+00, 2.36228715e-16,
       2.40706316e-01, 5.14030408e-16, 6.50541008e-16, 2.65609765e-01,
       0.00000000e+00, 3.60583905e-16])


In [ ]:
# The main function that UChicago will be running

def allocate_portfolio(asset_prices):
    
    # This simple strategy equally weights all assets every period
    # (called a 1/n strategy).
    
    n_assets = len(asset_prices)
    weights = np.repeat(1 / n_assets, n_assets)
    return weights